In [ ]:
import os
import shutil
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow 
import keras
from collections import deque
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

from sklearn.model_selection import train_test_split

import tensorflow as tf  
from keras.layers import *
from keras.models import load_model

In [ ]:
VIDEO_FRAME_HEIGHT = 64
VIDEO_FRAME_WIDTH = 64
FRAMES_IN_SEQUENCE = 16
DATASET_DIRECTORY = "./Dataset/Real Life Violence Dataset"
CLASS_LABELS = ["NonViolence","Violence"]

In [ ]:
my_model = load_model('Model.h5')

In [ ]:
def predict_video_class(video_file_path, FRAMES_IN_SEQUENCE=16):
    try:
        video_reader = cv2.VideoCapture(video_file_path)
        if not video_reader.isOpened():
            print("Error: Unable to open video file.")
            return
        video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
        frames_list = []
        skip_frames_window = max(int(video_frames_count / FRAMES_IN_SEQUENCE), 1)
        for frame_counter in range(FRAMES_IN_SEQUENCE):
            video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
            frame_found, frame = video_reader.read()
            if not frame_found:
                print("Error: Unable to read frame from video.")
                return
            resized_frame = cv2.resize(frame, (VIDEO_FRAME_HEIGHT, VIDEO_FRAME_WIDTH))
            normalized_frame = resized_frame / 255
            frames_list.append(normalized_frame)
        predicted_labels_probabilities = my_model.predict(np.expand_dims(frames_list, axis=0))[0]
        predicted_label_index = np.argmax(predicted_labels_probabilities)
        predicted_class_name = CLASS_LABELS[predicted_label_index]
        prediction_confidence = predicted_labels_probabilities[predicted_label_index]
        return (predicted_class_name, prediction_confidence)
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:
        if video_reader:
            video_reader.release()


In [ ]:
def alert(send=False,msg="Violence Detected"):
    if send:
        print (f"Alert: {msg}") 
    else:
        print ("Violence Detected")

In [ ]:
alert(True,"violence detected")

In [ ]:
def run(SEQUENCE_LENGTH=16):
    print("Running")
    video_reader = cv2.VideoCapture(0)
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # cc = cv2.VideoWriter_fourcc(*'XVID')
    cc = cv2.VideoWriter_fourcc(*'X264')
    file = cv2.VideoWriter('output.avi', cc, 15.0, (640, 480))
    # frames_queue = deque(maxlen = SEQUENCE_LENGTH)
    frames_queue = []
    predicted_class_name = ''
    predicted_label = 0
    cnt=0
    KPS = 10
    # fps = round(video_reader.get(cv2.CAP_PROP_FPS))
    curr_frame = 0
    send = True
    while video_reader.isOpened():
        try:
            ok, frame = video_reader.read() 
            if not ok:
                break
            if curr_frame % KPS == 0:
                resized_frame = cv2.resize(frame, (VIDEO_FRAME_HEIGHT, VIDEO_FRAME_WIDTH))
                normalized_frame = resized_frame / 255
                frames_queue.append(normalized_frame)
                name = "frame%d.jpg"%curr_frame
                cv2.imwrite(name, frame) 
            curr_frame += 1
            if len(frames_queue) == SEQUENCE_LENGTH:
                predicted_labels_probabilities = my_model.predict(np.expand_dims(frames_queue, axis = 0),verbose=0)[0]
                predicted_label = np.argmax(predicted_labels_probabilities)
                predicted_class_name = CLASS_LABELS[predicted_label] +' '+ str(float(predicted_labels_probabilities[predicted_label]))
                frames_queue.clear()
            if predicted_label == 1:
                cv2.putText(frame, predicted_class_name, (5, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                cnt+=1
            else:
                cv2.putText(frame, predicted_class_name, (5, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
                cnt=0
            if cnt == 10:
                alert(send)
                send = False
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            cv2.waitKey(1)
            cv2.imshow('Prediction', frame)
            file.write(frame)    
        except:
            break
    video_reader.release()
    file.release()
    cv2.destroyAllWindows() 
    print("Done")

In [ ]:
run()